In [1]:
import numpy as np
import torch
import torch.nn as nn

In [ ]:
class ActorNetwork(nn.Module):
    def __init__(self, input_dims, n_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_dims, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim = -1)

        return x

In [ ]:
class CriticNetwork(nn.Module):
    def __init__(self, input_dims):
        super().__init__()
        self.fc1 = nn.Linear(input_dims, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [ ]:
class Agent():
    def __init__(self, in_dims, n_actions):
        self.actor      = ActorNetwork(in_dims, n_actions)
        self.critic     = CriticNetwork(in_dims)
        self.actor_opt  = torch.optim.Adam(self.actor.paramters(),  lr = alpha)
        self.critic_opt = torch.optim.Adam(self.critic.paramters(), lr = beta)
        self.criterion  = nn.MSELoss()

    def learn(self):
        state, reward, action, next_state, probs, values, done = self.memory.Sample()
        new_probs = self.actor(state)
        advantage = 0
        store = []

        for i in reversed(range(self.size)):
            delta     = reward[i] + self.gamma * self.critic(next_state[i]) * (1 - int(done[i])) - values[i]

            advantage = delta + self.lembda * self.gamma * (advantage)

            store.append(advantage)
            

        store.reverse()

        store = np.array(store)

        store.reshape(len(state), 1)

        loss = store * (new_probs / probs)
